In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [3]:
# Store the CSV created in part one into a DataFrame.
vacation_df = pd.read_csv("weather_data/WeatherPy_vacation.csv")
vacation_df = vacation_df.set_index('City_ID')
vacation_df.head()

,City,Country,Current Description,Max Temp,Lat,Lng,Hotel Name
City_ID,,,,,,,
5,Mahebourg,MU,scattered clouds,71.60,-20.41,57.70,Shandrani Beachcomber Resort & Spa
16,Hilo,US,few clouds,84.20,19.73,-155.09,Hilo Hawaiian Hotel
17,Bathsheba,BB,scattered clouds,82.40,13.22,-59.52,Atlantis Hotel
22,Butaritari,KI,overcast clouds,82.69,3.07,172.79,Isles Sunset Lodge
24,Avarua,CK,scattered clouds,82.40,-21.21,-159.78,Paradise Inn


In [4]:
#Create DataFrames for the cities wanting to travel too
luorong_df = vacation_df.loc[(vacation_df["City"] == "Luorong")]
longhua_df = vacation_df.loc[(vacation_df["City"] == "Longhua")]
huilong_df = vacation_df.loc[(vacation_df["City"] == "Huilong")]
wanning_df = vacation_df.loc[(vacation_df["City"] == "Wanning")]

In [5]:
luorong_df

,City,Country,Current Description,Max Temp,Lat,Lng,Hotel Name
City_ID,,,,,,,
41,Luorong,CN,light rain,79.16,24.41,109.61,荣立大酒店


In [6]:
#Get the lat and lng pair for each city in a tuple
luorong_coord = ((float(luorong_df["Lat"])), (float(luorong_df["Lng"])))
longhua_coord = ((float(longhua_df["Lat"])), (float(longhua_df["Lng"])))
huilong_coord = ((float(huilong_df["Lat"])), (float(huilong_df["Lng"])))
wanning_coord = ((float(wanning_df["Lat"])), (float(wanning_df["Lng"])))

In [7]:
#Plot the direction path
fig = gmaps.figure()
vacation_path = gmaps.directions_layer(luorong_coord, wanning_coord, 
                                       waypoints=[huilong_coord, longhua_coord], travel_mode='DRIVING')
fig.add_layer(vacation_path)
fig

Figure(layout=FigureLayout(height='420px'))

In [8]:
#Use concat() function to create a new DataFrame that uses all 4 cities
vacation_itinerary_df = pd.concat([luorong_df, huilong_df, longhua_df, wanning_df])
vacation_itinerary_df

,City,Country,Current Description,Max Temp,Lat,Lng,Hotel Name
City_ID,,,,,,,
41,Luorong,CN,light rain,79.16,24.41,109.61,荣立大酒店
188,Huilong,CN,broken clouds,70.00,31.81,121.66,Hilton Garden Inn Qidong
439,Longhua,CN,overcast clouds,78.91,23.61,114.19,Huadu Hotel
134,Wanning,CN,overcast clouds,79.99,18.80,110.40,Paradise Hotel


In [9]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} & {Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in vacation_itinerary_df.iterrows()]

# Add a pop-up marker for each city.
locations = vacation_itinerary_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
vacation_path = gmaps.directions_layer(luorong_coord, wanning_coord, 
                                       waypoints=[huilong_coord, longhua_coord], travel_mode='DRIVING', show_markers=False)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
fig.add_layer(vacation_path)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))